# A [Jupyter](https://jupyter.org/) Kernel for [PowerShell](https://github.com/PowerShell/PowerShell)

This kernel is being written in C#, and in the process, I've taken some messaging code from the [iCSharp kernel](https://github.com/zabirauf/icsharp) and made a generic library for .Net with a re-usable core for anyone who needs to create [Jupyter](https://jupyter.org/) kernels in .Net languages -- so feel free to borrow that if you like (it's under the Apache license).

## Current Status

At this point, I'm only handling two messages:

* KernelInfo request
* Execute request

The PowerShell kernel is _working_, and returning text output _and errors_ as on the console (see examples below).

## Features

Apart from the built-in Jupyter features, I'm going to add some output enhancements so you can hook into widgets, etc. However, there's none of thata yet, except that:

* If you output HTML, it's rendered. I'm currently detecting this in the most simplistic fashion: by testing if the output starts with "<" and ends with ">". That probably needs work, but it's good enough for now.
* When a command outputs objects, you get the text rendering, but the actual objects are also returned as application/json data.

## PowerShell Core

In order to get cross-platform support, this kernel is using [PowerShell Core](https://github.com/PowerShell/PowerShell), which means you'll want to have PowerShell 6 Beta 3 installed to try it out. I'm hoping to provide a Full Framework (aka Windows PowerShell) version too, once I start doing releases.

To use it or build it, you need [dotnet core 2 preview](https://www.microsoft.com/net/core/preview), and if you want to contribute, and want to build it in Visual Studio, you need [VS 2017 Preview version 15.3](https://www.visualstudio.com/vs/preview/).

In [1]:
$PSVersionTable

Name                           Value                                                                                    
----                           -----                                                                                    
PSVersion                      6.0.0-beta                                                                               
PSEdition                      Core                                                                                     
GitCommitId                    Could not find file 'C:\Users\Joel\.nuget\packages\system.management.automation\6.0.0-...
OS                             Microsoft Windows 10.0.16215                                                             
Platform                       Win32NT                                                                                  
PSCompatibleVersions           {1.0, 2.0, 3.0, 4.0...}                                                                  
PSRemotingProtocolVersion      2

In [2]:
Get-ChildItem

Directory: C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell


Mode                LastWriteTime         Length Name                                                                   
----                -------------         ------ ----                                                                   
d-----        6/25/2017   1:50 AM                .ipynb_checkpoints                                                     
d-----        6/24/2017  12:35 AM                Jupyter                                                                
d-----        6/26/2017  10:53 PM                PowerShell-Kernel                                                      
-a----        6/16/2017  11:57 PM           2581 .gitattributes                                                         
-a----        6/25/2017   1:54 AM           4588 .gitignore                                                             
-a----        6/26/2017   5:05 PM           1874 Jupyter-PowerShell.sln                   

In [3]:
$Files = Get-ChildItem -File

## No Output.

Obviously there's no output from that command, because it's all collected in the variable.

I put that here was just to demonstrate that variables stick around so you can use them in other commands in the future, so let's look at the size of those files again:

In [4]:
$Files | Measure Length -Sum

Count    : 7
Average  : 
Sum      : 29906
Maximum  : 
Minimum  : 
Property : Length

In [5]:
Set-Location PowerShell-Kernel

Again, any command with no output just has ... no output.

### But it affects the state.

This means you will be able to use this for literate devops...

Oh, and since I mentioned you can do HTML output, here's PowerShell's built-in ConvertTo-Html

In [6]:
Get-ChildItem -file | ConvertTo-Html Mode, LastWriteTime, Name, Length

Mode,LastWriteTime,Name,Length
-a----,6/26/2017 10:42:55 PM,ExecutionResult.cs,1396
-a----,6/26/2017 10:53:38 PM,PowerShellEngine.cs,7157
-a----,6/27/2017 12:31:31 AM,PowerShell-Kernel.csproj,1503
-a----,6/26/2017 1:54:08 PM,Program.cs,1293
-a----,6/26/2017 1:54:08 PM,ScalarHelper.cs,2624


In [8]:
"<img src='https://upload.wikimedia.org/wikipedia/commons/2/2f/PowerShell_5.0_icon.png' />"

<img src='https://upload.wikimedia.org/wikipedia/commons/2/2f/PowerShell_5.0_icon.png' />

### I fixed [3](https://github.com/Jaykul/Jupyter-PowerShell/issues/3) so errors show up:

In [7]:
Get-ChildItem nosuchfile

PathNotFound,Microsoft.PowerShell.Commands.GetChildItemCommand: Get-ChildItem : Cannot find path 'C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\PowerShell-Kernel\nosuchfile' because it does not exist.
At line:1 char:1
+ Get-ChildItem nosuchfile
+ ~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (C:\Users\Joel\P...rnel\nosuchfile:String) [Get-ChildItem], ItemNotFoundException
    + FullyQualifiedErrorId : PathNotFound,Microsoft.PowerShell.Commands.GetChildItemCommand

Get-ChildItem : Cannot find path 'C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\PowerShell-Kernel\nosuchfile' because it does not exist.
At line:1 char:1
+ Get-ChildItem nosuchfile
+ ~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (C:\Users\Joel\P...rnel\nosuchfile:String) [Get-ChildItem], ItemNotFoundException
    + FullyQualifiedErrorId : PathNotFound,Microsoft.PowerShell.Commands.GetChildItemCommand